In [1]:
import spectfbcalc_lib as sfc
from climtools import climtools_lib as ctl

No DISPLAY variable set. Switching to agg backend


/home/rotoli/miniconda3/envs/climtools/lib/python3.13/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [ ]:
from climtools import climtools_lib as ctl
import coso as sfc

In [12]:
from importlib import reload
reload(sfc)

<module 'spectfbcalc_lib' from '/home/rotoli/codici/SpectFbCalc/spectfbcalc/spectfbcalc_lib.py'>

In [12]:
sfc.mytestfunction()

test!


In [3]:
ctl.datestamp()

'2024-11-12T13:22:36'

In [2]:
import sys
import os
import glob

import numpy as np
import xarray as xr

from climtools import climtools_lib as ctl
from matplotlib import pyplot as plt
import matplotlib.cbook as cbook
from scipy import stats
import pickle
import dask.array as da
import yaml
import pandas as pd


In [3]:
cart_out = '/home/rotoli/codici/test_nuovo/'
ctl.mkdir(cart_out)

cart_in = '/nas/archive_CMIP6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/'
filin_pi = cart_in + 'piControl/atmos/Amon/r1i1p1f1/{}/*nc'
filin_4c = cart_in + 'abrupt-4xCO2/atmos/Amon/r8i1p1f1_r25/{}/*nc'
filin_4c1 = cart_in + 'abrupt-4xCO2/atmos/Amon/r8i1p1f1/{}/*nc'
cart_k = '/data-hobbes/fabiano/radiative_kernels/Huang/toa/'
use_climatology=True
fb_coef=dict()
fb_coef2=dict()
#cart_k="/data-hobbes/fabiano/radiative_kernels/Huang_ERA5/"
dir='/data-hobbes/fabiano/radiative_kernels/ps_ece3/ps_Amon_EC-Earth3_stabilization-hist-1990_r1i1p1f1_gr_199?01-199?12.nc'


In [3]:
config_file='/home/rotoli/codici/SpectFbCalc/spectfbcalc/config_deb.yaml'
allkers=dict()
ker= 'HUANG'
standard_dict = sfc.standard_names

allkers=sfc.load_kernel_wrapper(ker, config_file)

In [6]:
with open(config_file, 'r') as file:
    config = yaml.safe_load(file)
time_range_clim = config.get("time_range", {})
time_range_clim = (time_range_clim.get("start"), time_range_clim.get("end"))
time_range_clim
cart_in = '/nas/archive_CMIP6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/'
filin_pi = cart_in + 'piControl/atmos/Amon/r1i1p1f1/{}/*nc'
piok=sfc.climatology(filin_pi, allkers, 'ts', time_range_clim, False, 12)

Regridding...
Regridding completed in 0:00:02.942815


In [2]:
ps_files="/data-hobbes/fabiano/radiative_kernels/ps_ece3/ps_Amon_EC-Earth3_stabilization-hist-1990_r1i1p1f1_gr_199?01-199?12.nc"
surf_pressure = xr.open_mfdataset(ps_files, combine='by_coords')
#vlevs=pickle.load(open( '/home/rotoli/codici/test_nuovamaskpress/vlevs_HUANG.p', 'rb'))
#mask=sfc.press_sup(surf_pressure, vlevs)

In [10]:
allvars = 'hus ta'.split()
piok=sfc.ref_clim(config_file, allvars, ker, standard_dict, allkers)
#pickle.dump(piok, open(cart_out + 'piok_huang.p', 'wb'))

Check that specified files in config have the necessary variables.
Standard variable names: ['rsus', 'rsds', 'time', 'lat', 'lon', 'plev', 'ps', 'ts', 'tas', 'ta', 'hus', 'rlut', 'rsut', 'rlutcs', 'rsutcs']
Not recognized variables that were not renamed: ['time_bnds', 'lat_bnds', 'lon_bnds']
Variables in dataset after renaming: ['time_bnds', 'lat_bnds', 'lon_bnds', 'hus', 'time', 'plev', 'lat', 'lon', 'ta']
Using pre-loaded kernels.
Regridding...
Regridding completed in 0:00:03.041701
Regridding...
Regridding completed in 0:00:01.121328


In [ ]:
ds=sfc.read_data(config_file, standard_dict)

In [ ]:
time_range=['1850-01-01', '1999-12-31']
rad=dict()
rad=sfc.Rad_anomaly_planck_surf(ds, piok, ker, allkers, cart_out, False, time_range, False)

In [55]:
cart_in = '/nas/archive_CMIP6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/'
filin_pi = cart_in + 'piControl/atmos/Amon/r1i1p1f1/{}/*nc'
piok=sfc.climatology(filin_pi, allkers, 'alb', None, True, 12)

Regridding...
Regridding completed in 0:00:00.937892
Regridding...
Regridding completed in 0:00:00.916653


In [ ]:
rad=dict()
ps_files="/data-hobbes/fabiano/radiative_kernels/ps_ece3/ps_Amon_EC-Earth3_stabilization-hist-1990_r1i1p1f1_gr_199?01-199?12.nc"
surf_pressure = xr.open_mfdataset(ps_files, combine='by_coords')
cart_out="/home/rotoli/codici/test_nuovamaskpress/"
piok=pickle.load(open('/home/rotoli/codici/test_provapress/piok_huang.p', 'rb'))
rad=sfc.Rad_anomaly_wv(ds, piok, 'HUANG', allkers, cart_out, surf_pressure, True, None, config_file, False)

In [6]:
piok=pickle.load(open('/home/rotoli/codici/test_provapress/piok_huang.p', 'rb'))
k=allkers[('cld', 't')]
var_tas= ctl.regrid_dataset(ds['tas'], k.lat, k.lon) 
fb_coef=dict()
fbnams = ['planck-surf', 'planck-atmo', 'lapse-rate', 'water-vapor', 'albedo']
anoms_tas = var_tas.groupby('time.month') - piok['tas']
cart_out="/home/rotoli/codici/test_nuovamaskpress/"
        
gtas = ctl.global_mean(anoms_tas).groupby('time.year').mean('time') 
gtas= gtas.groupby((gtas.year-1) // 10 * 10).mean()

for tip in ['clr', 'cld']:
        for fbn in fbnams:
            feedbacks=xr.open_dataarray(cart_out+"dRt_" +fbn+"_global_"+tip+ "_climatology-"+ker+"kernels.nc",  use_cftime=True)
            feedback=feedbacks.groupby((feedbacks.year-1) // 10 * 10).mean()

            res = stats.linregress(gtas, feedback)
            fb_coef[(tip, fbn)] = res
    

Regridding...
Regridding completed in 0:00:01.107480


/tmp/ipykernel_27579/1330576646.py:14: DeprecationWarning: Usage of 'use_cftime' as a kwarg is deprecated. Please pass a 'CFDatetimeCoder' instance initialized with 'use_cftime' to the 'decode_times' kwarg instead.
Example usage:
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
    ds = xr.open_dataset(decode_times=time_coder)

  feedbacks=xr.open_dataarray(cart_out+"dRt_" +fbn+"_global_"+tip+ "_climatology-"+ker+"kernels.nc",  use_cftime=True)
/tmp/ipykernel_27579/1330576646.py:14: DeprecationWarning: Usage of 'use_cftime' as a kwarg is deprecated. Please pass a 'CFDatetimeCoder' instance initialized with 'use_cftime' to the 'decode_times' kwarg instead.
Example usage:
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
    ds = xr.open_dataset(decode_times=time_coder)

  feedbacks=xr.open_dataarray(cart_out+"dRt_" +fbn+"_global_"+tip+ "_climatology-"+ker+"kernels.nc",  use_cftime=True)
/tmp/ipykernel_27579/1330576646.py:14: DeprecationWarning: Usage of 'use_cftime'

AttributeError: 'DataArray' object has no attribute 'year'

In [ ]:
fb_cloud, fb_cloud_err =sfc.feedback_cloud(ds, piok, fb_coef, gtas, None)

In [5]:
vlevs=pickle.load(open(cart_out + 'vlevs_HUANG.p', 'rb'))
k = allkers[('cld', 't')]

# If surf_pressure is an array:
if isinstance(surf_pressure, xr.Dataset):
    psclim = surf_pressure.groupby('time.month').mean(dim='time')
    psye = psclim['ps'].mean('month')

psye_rg = ctl.regrid_dataset(psye, k.lat, k.lon).compute()
wid_mask = np.empty([len(vlevs.player)] + list(psye_rg.shape))

for ila in range(len(psye_rg.lat)):
        for ilo in range(len(psye_rg.lon)):
            ind = np.where((psye_rg[ila, ilo].values/100. - vlevs.player.values) > 0)[0][0]
            wid_mask[:ind, ila, ilo] = np.nan
            wid_mask[ind, ila, ilo] = psye_rg[ila, ilo].values/100. - vlevs.player.values[ind]
            wid_mask[ind+1:, ila, ilo] = vlevs.dp.values[ind+1:]
        

wid_mask = xr.DataArray(wid_mask, dims = k.dims[1:], coords = k.drop('month').coords)


Regridding...
Regridding completed in 0:00:02.791176


/tmp/ipykernel_21675/624614627.py:20: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  wid_mask = xr.DataArray(wid_mask, dims = k.dims[1:], coords = k.drop('month').coords)


In [48]:
cart_k ='/data-hobbes/fabiano/radiative_kernels/Huang/toa/'
vlevs = xr.load_dataset( cart_k+'dp.nc')
vlevs.dp

<xarray.DataArray 'dp' (player: 24)> Size: 96B
array([ 25.5,  25. ,  25. ,  25. ,  25. ,  25. ,  25. ,  25. ,  25. ,
        25. ,  87.5, 100. , 100. , 100. , 100. ,  62.5,  50. ,  50. ,
        52.5,  35. ,  16.5,   4.5,   2.5,   1. ], dtype=float32)
Coordinates:
  * player   (player) float32 96B 1e+03 975.0 950.0 925.0 ... 15.0 6.0 2.5 1.0

In [ ]:
sp=(surf_pressure.sel(time='1990-01-16T12:00:00.000000000')).ps
nlayer = 24
nlat = sp.shape[0]
nlon = sp.shape[1]

player = vlevs["player"]     # [lev] in hPa
plevel = vlevs["plevel"]     # [lev+1] in hPa
dp = vlevs["dp"]             # [lev] in hPa

# Inizializza l'array dps con NaN (equiv. _FillValue)
dps = np.full((nlayer, nlat, nlon), np.nan)

# Loop su lat e lon
for i in range(nlat):
    for j in range(nlon):
        ps_ij = sp[i, j].values  # pressione superficiale in hPa

        # Trova primo livello sopra la superficie
        idx = np.where(ps_ij - player.values >= 0)[0][0]

        # Assegna il valore parziale al livello della superficie
        dps[idx, i, j] = ps_ij - plevel[idx + 1]

        # Assegna i dp standard sopra la superficie
        dps[idx + 1:, i, j] = dp[idx + 1:].values

# Converti in DataArray
dps_xr = xr.DataArray(dps, dims=("player", "lat", "lon"),
                      coords={"player": player, "lat": sp.lat, "lon": sp.lon})

In [ ]:
dps_xr

<xarray.DataArray 'player' (player: 24)> Size: 96B
array([1000. ,  975. ,  950. ,  925. ,  900. ,  875. ,  850. ,  825. ,  800. ,
        775. ,  725. ,  625. ,  525. ,  425. ,  325. ,  237.5,  187.5,  137.5,
         85. ,   40. ,   15. ,    6. ,    2.5,    1. ], dtype=float32)
Coordinates:
  * player   (player) float32 96B 1e+03 975.0 950.0 925.0 ... 15.0 6.0 2.5 1.0
Attributes:
    units:    hPa

In [9]:
# nuovo=ctl.regrid_dataset(dps_xr, k.lat, k.lon)
# diff=wid_mask.sel(player=850) #- nuovo.sel(player=850)

import cartopy.crs as ccrs
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
surf_pressure
ciao.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="RdBu", cbar_kwargs={"label": "Differenza pressione (hPa)"})
ax.coastlines()
ax.set_title(f"press")
plt.savefig('press wid_mask 850')
plt.show()

In [10]:
mask_prs=sfc.mask_pres(surf_pressure, cart_out, allkers, config_file)
mask_prs

Regridding...
Regridding completed in 0:00:01.016419


<xarray.DataArray (player: 24, lat: 73, lon: 144)> Size: 2MB
array([[[14.14849854, 14.14849854, 14.14849854, ..., 14.14849854,
         14.14849854, 14.14849854],
        [13.78869629, 13.8079834 , 13.82727051, ..., 13.49407959,
         13.59234619, 13.69055176],
        [13.22076416, 13.24700928, 13.27337646, ..., 13.38470459,
         13.25408936, 13.23736572],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[25.        , 25.        , 25.        , ..., 25.        ,
         25.        , 25.        ],
        [25.        , 25.        , 25.        , ..., 25.        ,
         25.        , 25.        ],
        [25.        , 25.        , 25.        , ..., 25.        ,
         25.        , 25.        ],
...
        [ 2.5       ,  2.5       ,  2.5       , ...,  2.5       ,
          2.5       ,  2.5       ],
        [ 2.5       ,  2.5       ,  2.5       , ...,  2.5       ,
          2.5       ,  2.5       ],
        [ 2.5       ,  2.5       ,  2.5       , ...,  2.5       ,
          2.5       ,  2.5       ]],

       [[ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        ...,
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ]]])
Coordinates:
  * player   (player) float32 96B 1e+03 975.0 950.0 925.0 ... 15.0 6.0 2.5 1.0
  * lat      (lat) float32 292B 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 576B 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5

In [11]:
mask_atm=sfc.mask_atm(ds['ta'])
mask_atm

<xarray.DataArray (time: 1980, plev: 18, lat: 256, lon: 512)> Size: 37GB
dask.array<truediv, shape=(1980, 18, 256, 512), dtype=float64, chunksize=(12, 9, 128, 256), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * plev     (plev) float64 144B 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat      (lat) float64 2kB -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon      (lon) float64 4kB 0.0 0.7031 1.406 2.109 ... 357.9 358.6 359.3
    height   float64 8B 2.0

In [9]:

anom=dict()
ps_files="/data-hobbes/fabiano/radiative_kernels/ps_ece3/ps_Amon_EC-Earth3_stabilization-hist-1990_r1i1p1f1_gr_199?01-199?12.nc"
surf_pressure = xr.open_mfdataset(ps_files, combine='by_coords')
cart_out="/home/rotoli/codici/test_nuovo/"
cart_k="/data-hobbes/fabiano/radiative_kernels/Huang/toa/"
finam= "RRTMG_{}_toa_{}_highR.nc"
allkers=dict()
allkers=sfc.load_kernel('HUANG', cart_k, cart_out, finam)
print('fatto kernel')
piok=pickle.load(open(cart_out + 'piok_HUANG.p', 'rb'))
#anom=sfc.calc_anoms(ds, piok, 'HUANG', allkers, cart_out, surf_pressure)
#anom=sfc.calc_anoms_wrapper(config_file, ker, standard_dict)

fatto kernel


In [10]:
fb_coef=dict()
fb_coef, fb_cloud, fb_cloud_err =sfc.calc_fb(ds, piok, ker, allkers, cart_out, surf_pressure, True, None, False, config_file)


_climatology
planck surf
planck atm
albedo
w-v
Regridding...
Regridding completed in 0:00:03.063162
calcolo feedback


In [12]:
fb_cloud

np.float64(-1.7474654652328225)

In [42]:
anom=sfc.Rad_anomaly_planck_atm_lr(ds,piok, ker, allkers, cart_out, surf_pressure, True, None, None, False)

Regridding...
Regridding completed in 0:00:00.968936
Regridding...
Regridding completed in 0:00:01.200634
Regridding...
Regridding completed in 0:00:01.186864


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  


Processing clr
Kernel loaded successfully


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  


Processing cld
Kernel loaded successfully


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  


In [35]:
anom=sfc.Rad_anomaly_planck_surf_wrapper(config_file, ker, standard_dict)

Kernel upload...
Dataset to analyze upload...


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/dask/array/core.py:4831: PerformanceWarning: Increasing number of chunks by factor of 19
  result = blockwise(
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/dask/array/core.py:4831: PerformanceWarning: Increasing number of chunks by factor of 19
  result = blockwise(


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    # compilation of odd code (such as a naked 'return' outside a
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_10188/540591705.py", line 1, in <module>
    anom=sfc.Rad_anomaly_planck_surf_wrapper(config_file, ker, standard_dict)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rotoli/codici/SpectFbCalc/spectfbcalc/spectfbcalc_lib.py", line 738, in Rad_anomaly_planck_surf_wrapper
    ds = read_data(config_file, standard_names)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rotoli/codici/SpectFbCalc/spectfbcalc/spectfbcalc_lib.py", line 277, in read_data
    ds_list = [xr.open_mfdataset(file_path1, combine='by_coords', use_cftime=True, chunks={'time': time_chunk})]
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
mask =sfc.mask_atm(ds['ta'])
mask

<xarray.DataArray (time: 1980, plev: 18, lat: 329, lon: 640)>
dask.array<truediv, shape=(1980, 18, 329, 640), dtype=float64, chunksize=(12, 1, 329, 640), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lon      (lon) float64 0.0 0.7031 1.406 2.109 ... 357.5 357.9 358.6 359.3
  * lat      (lat) float64 -90.0 -89.46 -88.77 -88.07 ... 88.07 88.77 89.46 90.0
  * plev     (plev) float64 9.25e+04 8.5e+04 7e+04 6e+04 ... 1e+03 500.0 100.0
    height   float64 2.0

In [14]:
piok

<xarray.Dataset> Size: 76MB
Dimensions:  (time: 1800, lat: 73, lon: 144)
Coordinates:
  * time     (time) object 14kB 2540-01-16 12:00:00 ... 2689-12-16 12:00:00
  * lat      (lat) float32 292B 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 576B 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    ts       (time, lat, lon) float32 76MB dask.array<chunksize=(258, 1, 2), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [15]:
ker='HUANG'
use_climatology=True
use_ds_climatology = False
time_range =None
piok=pickle.load(open(cart_out + 'piok_HUANG.p', 'rb'))
ps_files="/data-hobbes/fabiano/radiative_kernels/ps_ece3/ps_Amon_EC-Earth3_stabilization-hist-1990_r1i1p1f1_gr_199?01-199?12.nc"
surf_pressure = xr.open_mfdataset(ps_files, combine='by_coords')

if ker == 'HUANG' and surf_pressure is None:
        raise ValueError("Error: The 'surf_pressure' parameter cannot be None when 'ker' is 'HUANG'.")

radiation=dict()
k= allkers[('cld', 't')]
cose=pickle.load(open(cart_out + 'cose_'+ker+'.p', 'rb'))

if ker=='HUANG':
        wid_mask=sfc.mask_pres(surf_pressure, cart_out, allkers, config_file)

   
if use_climatology==True:
        cos="_climatology"
else:
        cos="_21yearmean"

if time_range is not None:
        print('è stronzo')
        var = ds['ta'].sel(time = slice(time_range[0], time_range[1])) 
        var_ts = ds['ts'].sel(time = slice(time_range[0], time_range[1])) 
        var=ctl.regrid_dataset(var, k.lat, k.lon)
        var_ts=ctl.regrid_dataset(var_ts, k.lat, k.lon)
else:
        var = ctl.regrid_dataset(ds['ta'], k.lat, k.lon)
        var_ts = ctl.regrid_dataset(ds['ts'], k.lat, k.lon)

if use_ds_climatology == False:
        print('ciclo giusto 1')
        if use_climatology==False:
            sfc.check_data(var, piok['ta'])
            piok_ta=piok['ta'].drop('time')
            piok_ta['time'] = var['time']
            piok_ts=piok['ts'].drop('time')
            piok_ts['time'] = var['time']
            anoms_ok = var - piok_ta
            ts_anom = var_ts - piok_ts
        else:
            print('calcola anoms nel posto giusto')
            anoms_ok=var.groupby('time.month') - piok['ta']
            ts_anom=var_ts.groupby('time.month') - piok['ts']
else: 
        if use_climatology==False:
            sfc.check_data(ds['ta'], piok['ta'])
            piok_ta=piok['ta'].drop('time')
            piok_ta['time'] = var['time']
            piok_ts=piok['ts'].drop('time')
            piok_ts['time'] = var['time']
            anoms_ok = var.groupby('time.month').mean() - piok_ta.groupby('time.month').mean()
            ts_anom = var_ts.mean('time') - piok_ts.mean('time')
        else:
            anoms_ok=var.groupby('time.month').mean() - piok['ta']
            ts_anom = var_ts.groupby('time.month') - piok['ts']
            ts_anom = ts_anom.compute()

if use_ds_climatology == False:
        mask=sfc.mask_atm(var)
else:
        mask=sfc.mask_atm(var)
        mask = mask.groupby('time.month').mean()

anoms_ok = (anoms_ok*mask).interp(plev = cose) 

for tip in ['clr','cld']:
        print(f"Processing {tip}")  
        try:
            kernel = allkers[(tip, 'ts')]
            print("Kernel loaded successfully")  
        except Exception as e:
            print(f"Error loading kernel for {tip}: {e}")  
            continue  

        if ker=='HUANG':
            if use_ds_climatology == False:
                anoms_lr = (anoms_ok - ts_anom)  
                anoms_unif = (anoms_ok - anoms_lr)
            else: 
                anoms_lr = (anoms_ok - ts_anom.mean('time'))
                anoms_unif = (anoms_ok - anoms_lr)
        if ker=='ERA5':
            if use_ds_climatology == False:
                anoms_lr = (anoms_ok - ts_anom)  
                anoms_unif = (anoms_ok - anoms_lr)
            else: 
                anoms_lr = (anoms_ok - ts_anom.groupby('time.month').mean())
                anoms_unif = (anoms_ok - anoms_lr)
                print("anoms_lr:", anoms_lr.sizes) #DA CONTROLLARE!
                print("dims:", anoms_lr.dims)
                print("anoms_lr:", anoms_ok.sizes)
                print("dims:", anoms_ok.dims)

        if ker=='HUANG':
            if use_ds_climatology == False:
                dRt_unif = (anoms_unif.groupby('time.month')*kernel*wid_mask/100).sum('player').groupby('time.year').mean('time')  
                dRt_lr = (anoms_lr.groupby('time.month')*kernel*wid_mask/100).sum('player').groupby('time.year').mean('time')   
            else:
                dRt_unif = (anoms_unif*kernel*wid_mask/100.).sum('player').mean('month')  
                dRt_lr = (anoms_lr*kernel*wid_mask/100.).sum('player').mean('month')  

        dRt_unif_glob = ctl.global_mean(dRt_unif)
        dRt_lr_glob = ctl.global_mean(dRt_lr)
        feedbacks_atmo = dRt_unif_glob.compute()
        feedbacks_lr = dRt_lr_glob.compute()
        radiation[(tip,'planck-atmo')]=feedbacks_atmo
        radiation[(tip,'lapse-rate')]=feedbacks_lr 
    

Regridding...
Regridding completed in 0:00:01.149438
Regridding...
Regridding completed in 0:00:01.134938
Regridding...
Regridding completed in 0:00:01.142155
ciclo giusto 1
calcola anoms nel posto giusto
Processing clr
Kernel loaded successfully
Processing cld
Kernel loaded successfully


In [23]:
ts_anom

<xarray.DataArray (time: 1980, lat: 73, lon: 144)> Size: 83MB
dask.array<sub, shape=(1980, 73, 144), dtype=float32, chunksize=(1, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    height   float64 8B 2.0
  * lat      (lat) float32 292B 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 576B 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    month    (time) int64 16kB 1 2 3 4 5 6 7 8 9 10 ... 3 4 5 6 7 8 9 10 11 12

In [17]:
feedbacks_atmo

<xarray.DataArray (year: 165)> Size: 1kB
array([ -6.86172691,  -9.60538153, -11.91936601, -15.04975936,
       -16.17735836, -16.92984257, -19.26654205, -19.03680721,
       -19.11753208, -20.12017843, -21.03292371, -21.63884781,
       -21.20725205, -22.56497874, -22.9830102 , -21.25538553,
       -21.86660681, -23.43630519, -23.67377371, -24.13439483,
       -24.93499651, -24.36907819, -24.84839176, -26.05487461,
       -25.05301581, -25.50769659, -26.60349511, -26.53323464,
       -24.78792101, -25.06375832, -26.79285942, -26.93299189,
       -27.00073929, -26.13228705, -25.79771899, -27.87566241,
       -27.28819083, -25.91310286, -27.17307137, -28.39783047,
       -27.93002367, -28.87563292, -28.30502699, -28.5368622 ,
       -29.56066384, -27.88833642, -27.62328102, -28.79113549,
       -29.88744778, -27.53095934, -29.04229936, -30.91606679,
       -30.96264065, -29.64732695, -29.8333411 , -31.11820993,
       -31.03636524, -29.8349755 , -30.6353857 , -31.47126129,
       -30.21665426, -30.29573563, -30.55749398, -31.22812213,
       -31.6065401 , -30.91296156, -30.21373793, -31.4946438 ,
       -32.89180707, -31.33236345, -31.74347466, -33.3607068 ,
       -32.62348657, -32.49714349, -32.80930607, -32.28336783,
       -31.34283317, -31.7598167 , -33.13535172, -33.57074024,
...
       -31.66712176, -33.41790318, -34.06719343, -33.01718088,
       -33.76809202, -34.21034244, -33.84736121, -33.66735627,
       -33.51559304, -34.77824251, -34.86700707, -32.13503673,
       -33.2914315 , -34.45499529, -34.94313333, -33.09654218,
       -33.89479471, -33.778489  , -33.58336363, -34.46141395,
       -33.15945558, -33.12146479, -33.90623245, -34.38652874,
       -34.15191053, -33.79238809, -33.55376454, -34.11229318,
       -35.57209398, -34.60686862, -34.43148953, -34.592162  ,
       -34.34806722, -34.3799013 , -35.01997646, -34.33467551,
       -33.577759  , -34.37230801, -35.29321347, -34.18930115,
       -34.32713682, -36.16902367, -35.5082687 , -34.46750096,
       -35.20011898, -35.01621283, -35.21040891, -34.71816714,
       -34.79679329, -35.82134051, -35.29284217, -34.87296958,
       -34.27755683, -35.22856197, -36.15293497, -36.39050797,
       -36.85156703, -34.24319968, -34.66252452, -36.17671045,
       -35.77840713, -34.26073201, -34.80974112, -36.34291462,
       -35.26415469, -35.01656768, -36.02324616, -35.75022332,
       -34.85967718, -35.29905056, -36.38228073, -36.72022911,
       -35.7020837 , -36.26581608, -36.26712863, -37.12749817,
       -35.45494274])
Coordinates:
    height   float64 8B 2.0
  * year     (year) int64 1kB 1850 1851 1852 1853 1854 ... 2011 2012 2013 2014

In [17]:
res.slope #+ fb_coef[('cld', 'planck-surf')].slope

np.float64(-4.956454599270212)

In [20]:
fb_coef[('cld', 'planck-surf')]

LinregressResult(slope=np.float64(-1.9704261932790594), intercept=np.float64(2.0541213010637396), rvalue=np.float64(-0.9905592083414911), pvalue=np.float64(3.080528738432175e-15), stderr=np.float64(0.06817285748402739), intercept_stderr=np.float64(0.41933261463657023))

In [7]:
from matplotlib import pyplot as plt
fig=plt.figure(figsize=(7,5))
rad.plot()
plt.savefig(cart_out+'prova.pdf')

In [ ]:
rad=sfc.Rad_anomaly_wv(ds, piok, 'HUANG', allkers, cart_out, surf_pressure, True, None, config_file, False)

In [35]:
rad=dict()
rad=sfc.Rad_anomaly_albedo(ds, piok, 'HUANG', allkers, cart_out, True, None, False)

Regridding...
Regridding completed in 0:00:01.228515


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in 

In [ ]:
time_range=['2540-01-01', '2689-12-31']
piok=dict()
allvars= 'ts rsus rsds ta tas hus'.split()
for vnams in allvars:  
        piok[vnams]=sfc.climatology(filin_pi, allkers, vnams, time_range, False, 4)
allvars1= 'rlutcs rsutcs rlut rsut'.split()
for vnams in allvars1:  
        piok[vnams]=sfc.climatology(filin_pi, allkers, vnams)
# piok=xr.merge(piok.values())

In [ ]:
pickle.dump(piok, open(cart_out + 'piok_HUANG.p', 'wb'))

In [6]:
piok=pickle.load(open(cart_out + 'piok_HUANG.p', 'rb'))


In [ ]:
use_ds_climatology=True
use_climatology=False
time_range=['1850-01-01', '1999-12-31']
radiation=dict()
k=allkers[('cld', 't')]
if time_range is not None:
        var = ds_all['ts'].sel(time = slice(time_range[0], time_range[1]))
var = ctl.regrid_dataset(var, k.lat, k.lon)
if use_ds_climatology == False:
    print('non doveva entrare in questo ciclo')
    if use_climatology == False:
        sfc.check_data(var, piok['ts'])
        piok = piok['ts'].drop('time')
        piok['time'] = var['time']
        piok = piok.chunk(var.chunks)
        anoms = var - piok
    else:
        anoms = var.groupby('time.month') - piok['ts']
else:
    if use_climatology == False:
        sfc.check_data(var, piok['ts'])
        piok = piok['ts'].drop('time')
        piok['time'] = var['time']
        piok = piok.chunk(var.chunks)
        anoms = var.groupby('time.month').mean() - piok.groupby('time.month').mean()
    else:
       print(f"ok 2 else")
       anoms = var.groupby('time.month').mean() - piok['ts']
for tip in ['clr', 'cld']:
        print(f"ok for")
        kernel = allkers[(tip, 'ts')]
        if use_ds_climatology == False:
           dRt = (anoms.groupby('time.month') * kernel).groupby('time.year').mean('time')
        else: 
            dRt = (anoms* kernel).mean('month')
        dRt_glob = ctl.global_mean(dRt)
        planck = dRt_glob.compute()
        radiation[(tip, 'planck-surf')] = planck
        planck.to_netcdf(cart_out + "dRt_planck-surf_global_" + tip + cos + "-" + ker + "kernels.nc", format="NETCDF4") 

Regridding...
Regridding completed in 0:00:00.172726
ok for
ok for


In [ ]:
fb_coef, fb_cloud, fb_cloud_err =sfc.calc_fb_wrapper(config_file, 'HUANG', standard_dict)

In [7]:

fb_coef, fb_cloud, fb_cloud_err =sfc.calc_fb(ds_all, piok, 'HUANG', allkers, cart_out, pressure)

_climatology
planck surf
planck atm
albedo
w-v
Regridding...
Regridding completed in 0:00:00.194012


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]


calcolo feedback


In [9]:
print(cart_out)

/home/rotoli/codici/test3/


In [8]:

time_range=['1850-01-01', '1999-12-31']
piok2=pickle.load(open(cart_out + 'piok_HUANG(21year).p', 'rb'))
fb_coef2, fb_cloud2, fb_cloud_err2 =sfc.calc_fb(ds_all, piok2, 'HUANG', allkers, cart_out, pressure, False, time_range)

planck surf
planck atm
albedo
w-v
Regridding...
Regridding completed in 0:00:00.191077


ValueError: conflicting sizes for dimension 'time': length 1800 on <this-array> and length 1980 on {'lat': 'lat', 'lon': 'lon', 'time': 'time'}

In [ ]:
ker= 'ERA5'
cart_k="/data-hobbes/fabiano/radiative_kernels/Huang_ERA5/"

fb_coef=sfc.calc_fb(filin_4c, filin_pi, cart_out, ker, cart_k, dir, True, 12)

In [10]:
piok2

{'ts': <xarray.DataArray (time: 1800, lat: 73, lon: 144)>
 dask.array<truediv, shape=(1800, 73, 144), dtype=float32, chunksize=(258, 73, 144), chunktype=numpy.ndarray>
 Coordinates:
   * time     (time) object 2540-01-16 12:00:00 ... 2689-12-16 12:00:00
   * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
   * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
 Attributes:
     regrid_method:  bilinear,
 'tas': <xarray.DataArray (time: 1800, lat: 73, lon: 144)>
 dask.array<truediv, shape=(1800, 73, 144), dtype=float32, chunksize=(258, 73, 144), chunktype=numpy.ndarray>
 Coordinates:
   * time     (time) object 2540-01-16 12:00:00 ... 2689-12-16 12:00:00
     height   float64 2.0
   * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
   * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
 Attributes:
     regrid_method:  bilinear,
 'hus': <xarray.DataArray (time: 1800, plev: 19, lat: 73, lon

In [5]:
fb_coef2=dict()
fb_coef2=sfc.calc_fb(filin_4c, filin_pi, cart_out, 'HUANG', cart_k, dir, True, 12)

_climatology
planck surf
planck atm
albedo
w-v
calcolo feedback


In [22]:
cos="_climatology"
Rv = 487.5 # gas constant of water vapor
Lv = 2.5e+06 # latent heat of water vapor
ta_abs_pi=xr.open_dataarray(cart_out+"ta_abs_pi"+cos+"-"+ker+"kernels.nc",  chunks = {'time': 12},  use_cftime=True)
gioco=( sfc.dlnws(ta_abs_pi) -((ta_abs_pi**2) * Rv/Lv))
sec=(ctl.global_mean(sfc.dlnws(ta_abs_pi)[3]))
pr=(ctl.global_mean((((ta_abs_pi**2) * Rv/Lv))[3]))
gra=ctl.global_mean(gioco[3])

In [36]:
valori = np.arange(210, 291)
temp = xr.DataArray(valori, dims=["ta"], coords={"ta": np.arange(len(valori))})
sec=(sfc.dlnws(temp))
pr=(((temp**2) * Rv/Lv))
gra=pr-sec

In [37]:
fig = plt.figure(figsize=(7,5))
gra.plot(label='diff')
sec.plot(label='dlnws')
pr.plot(label='t^2')
plt.legend()
plt.savefig('prova dlnws (valori lineari).pdf')

In [10]:
#cart_out = '/home/rotoli/codici/test5/'
#grafico feedback
from matplotlib import pyplot as plt

#ac2 = (fb_coef2[('cld', 'planck-surf')].slope + fb_coef2[('cld', 'planck-atmo')].slope)
#data2 = [ac2, fb_coef2[('cld','lapse-rate')].slope, fb_coef2[('cld', 'water-vapor')].slope, fb_coef2[('cld', 'albedo')].slope, fb_cloud2]
ac = (fb_coef[('cld', 'planck-surf')].slope + fb_coef[('cld', 'planck-atmo')].slope)
data = [ac, fb_coef[('cld','lapse-rate')].slope, fb_coef[('cld', 'water-vapor')].slope, fb_coef[('cld', 'albedo')].slope]#, fb_cloud]
#err2 = [(fb_coef2[('cld', 'planck-surf')].stderr + fb_coef2[('cld', 'planck-atmo')].stderr), fb_coef2[('cld','lapse-rate')].stderr, fb_coef2[('cld', 'water-vapor')].stderr, fb_coef2[('cld', 'albedo')].stderr, fb_cloud_err2]
data1 =[-3.24, -0.22,  1.72,  0.58]#, 0.29]
err = [(fb_coef[('cld', 'planck-surf')].stderr + fb_coef[('cld', 'planck-atmo')].stderr), fb_coef[('cld','lapse-rate')].stderr, fb_coef[('cld', 'water-vapor')].stderr, fb_coef[('cld', 'albedo')].stderr]#, fb_cloud_err]
err1 = [0.05, 0.18, 0.14, 0.09]#, 0.36]
fbnams1 = ['planck', 'lapse-rate', 'water-vapor', 'albedo']#, 'cloud']

fig = plt.figure(figsize=(7,5))
offset=0.05
plt.errorbar(range(len(fbnams1)), data, yerr=err, marker=".", linestyle= 'None', label='Huang kernels (using climatology)', color='royalblue')
plt.errorbar([x+offset for x in range(len(fbnams1))], data1, yerr=err1, marker=".", linestyle= 'None', label='by Zelinka et al. 2020', color='navy')
#plt.errorbar([x-offset for x in range(len(fbnams1))], data2, yerr=err2, marker=".", linestyle= 'None', label='Huang kernels (using 21year mean)', color='b')
plt.xticks(range(len(fbnams1)), fbnams1)
plt.legend(loc='lower right')
plt.ylabel('[W $m^{-2}$ $K^{-1}$]')

plt.savefig(cart_out +'feedback_HUANG(climatology_provapresssup).pdf')

In [12]:
fb_coef[('cld', 'planck-atmo')]

LinregressResult(slope=np.float64(9.358520419235798e-07), intercept=np.float64(-1.2225736153652157e-06), rvalue=np.float64(0.9818316720109332), pvalue=np.float64(5.639210063870164e-13), stderr=np.float64(4.5216925201545196e-08), intercept_stderr=np.float64(2.781302144336936e-07))

In [15]:
fb_coef[('cld', 'planck-atmo')].slope

np.float64(2.1660273316125806e-08)